# Pong Tutorial

See https://kivy.org/docs/tutorials/pong.html for the original. 

First off, you'll need to make sure that you have Kivy installed. 

**TODO** Insert instructions here for where to go for Kivy installation instructions. 

**NEXT** Basic notebook instructions.

## Getting Started

Let’s start by getting a really simple Kivy app up and running. Create a directory for the game and a file named main.py

We recommend creating **your own version** of the pong tutorial, where you type in all of the code from this notebook into your own files as your run through the tutorial.  

Recall that the cell magic 

    !stuff
    
is equivalent to running 

    stuff

at the command line. 


In [ ]:
# show the directory I'm in
!pwd

We'll use the "cell magic" 
    
    %%file filename.extension

at the top of the cell to write the contents of the cell to `filename.extension`. 

In [ ]:
%%file main.py

from kivy.app import App
from kivy.uix.widget import Widget

class PongGame(Widget):
    pass

class PongApp(App):
    def build(self):
        return PongGame()
    
if __name__ == '__main__':
    PongApp().run()

Go ahead and run the application. It should just show a black window at this point. What we’ve done is create a very simple Kivy [`App`](https://kivy.org/docs/api-kivy.app.html#kivy.app.App), which creates an instance of the `PongGame` Widget class and returns it as the root element for the applications UI, which you should imagine at this point as a hierarchical tree of Widgets. Kivy places this widget-tree in the default Window. 

In [ ]:
# run main.py
!python main.py

## Add Simple Graphics

In the next step, we will draw the Pong background and scores by defining how the PongGame widget looks.

We will use a .kv file to define the look and feel of the PongGame class. Since our [`App`](https://kivy.org/docs/api-kivy.app.html#kivy.app.App) class is called `PongApp`, we can simply create a file called `pong.kv` in the same directory that will be automatically loaded when the application is run. So create a new file called ``pong.kv`` and add the following contents. (The contents of the cell will be explained below). 

In [ ]:
%%file pong.kv

#:kivy 1.0.9

<PongGame>:
    canvas:
        Rectangle:
            pos: self.center_x - 5, 0
            size: 10, self.height               
    Label:
        font_size: 70  
        center_x: root.width / 4
        top: root.top - 50
        text: "0"
        
    Label:
        font_size: 70  
        center_x: root.width * 3 / 4
        top: root.top - 50
        text: "0"

**COMMON ERROR**: The name of the kv file, e.g. pong.kv, must match the name of the app, e.g. PongApp (the part before the App ending).


Run the app again now, you should see a vertical bar in the middle, and two zeros where the player scores will be displayed.

In [ ]:
!python main.py

## Explaining the Kv File Syntax

Before going on to the next step, you might want to take a closer look at the contents of the kv file we just created and figure out what is going on. If you understand what’s happening, you can probably skip ahead to the next step.

On the very first line we have:
```python
#:kivy 1.0.9
```
This first line is required in every kv file. It should start with `#:kivy` followed by a space and the Kivy version it is intended for (so Kivy can make sure you have at least the required version, or handle backwards compatibility later on).

After that, we begin defining rules that are applied to all `PongGame` instances:
```python
    <PongGame>:
        ...
```
Like Python, kv files use indentation to define nested blocks. A block defined with a class name inside the `<` and `>` characters is a [`Widget`](https://kivy.org/docs/api-kivy.uix.widget.html#kivy.uix.widget.Widget) rule. It will be applied to any instance of the named class. If you replaced `PongGame` with `Widget` in our example, all Widget instances would have the vertical line and the two Label widgets inside them because it would define these rules for all Widget instances.


Inside a rule section, you can add various blocks to define the style and contents of the widgets they will be applied to. You can:

* set property values,
* add child widgets
* define a *canvas*section in which you can add Graphics instructions that define how the widget is rendered.

The first block inside the `<PongGame>` rule we have is a *canvas* block:

```python
<PongGame>:
    canvas:
        Rectangle:
            pos: self.center_x - 5, 0
            size: 10, self.height
```

So this canvas block says that the PongGame widget should draw some graphics primitives. In this case, we add a rectangle to the canvas. We set the *pos* of the rectangle to be 5 pixels left of the horizontal center of the widget, and 0 for *y*. The size of the rectangle is set to 10 pixels in width, and the widget’s height in height. The nice thing about defining the graphics like this, is that the rendered rectangle will be automatically updated when the properties of any widgets used in the value expression change.

**Note**
Try to resize the application window and notice what happens. That’s right, the entire UI resizes automatically. The standard behaviour of the Window is to resize an element based on its property *size_hint*. The default widget size_hint is (1,1), meaning it will be stretched 100% in both x-direction and y-direction and hence fill the available space. Since the *pos* and *size* of the rectangle, together with the *center_x* and *top* of the score labels were defined within the context of the `PongGame` class, these properties will automatically update when the corresponding widget properties change. Using the Kv language gives you automatic property binding. :)


The last two sections we add look pretty similar. Each of them adds a `Label` widget as a child widget to the `PongGame` widget. For now, the text on both of them is just set to “0”. We’ll hook that up to the actual score once we have the logic implemented, but the labels already look good since we set a bigger *font_size*, and positioned them relatively to the root widget. The `root` keyword can be used inside the child block to refer back to the parent/root widget the rule applies to (`PongGame` in this case):
```python
<PongGame>:
    # ...

    Label:
        font_size: 70
        center_x: root.width / 4
        top: root.top - 50
        text: "0"

    Label:
        font_size: 70
        center_x: root.width * 3 / 4
        top: root.top - 50
        text: "0"
```

## Add the Ball
Ok, so we have a basic pong arena to play in, but we still need the players and a ball to hit around. Let’s start with the ball. We’ll add a new `PongBall class` to create a widget that will be our ball and make it bounce around.

Here is the Python code for the `PongBall` class (to be added to `main.py`):
```python

from kivy.properties import NumericProperty, ReferenceListProperty
from kivy.vector import Vector

class PongBall(Widget):

    # velocity of the ball on x and y axis
    velocity_x = NumericProperty(0)
    velocity_y = NumericProperty(0)

    # referencelist property so we can use ball.velocity as
    # a shorthand, just like e.g. w.pos for w.x and w.y
    velocity = ReferenceListProperty(velocity_x, velocity_y)

    # ``move`` function will move the ball one step. This
    #  will be called in equal intervals to animate the ball
    def move(self):
        self.pos = Vector(*self.velocity) + self.pos
```

To make it all work, you also have to add the imports for the [Properties](https://kivy.org/docs/api-kivy.properties.html) `Property` classes used and the [`Vector`](https://kivy.org/docs/api-kivy.vector.html#kivy.vector.Vector).

Altogether that gives:

In [ ]:
%%file main.py

from kivy.app import App
from kivy.uix.widget import Widget
from kivy.properties import NumericProperty, ReferenceListProperty
from kivy.vector import Vector

class PongBall(Widget):

    # velocity of the ball on x and y axis
    velocity_x = NumericProperty(0)
    velocity_y = NumericProperty(0)

    # referencelist property so we can use ball.velocity as
    # a shorthand, just like e.g. w.pos for w.x and w.y
    velocity = ReferenceListProperty(velocity_x, velocity_y)

    # ``move`` function will move the ball one step. This
    #  will be called in equal intervals to animate the ball
    def move(self):
        self.pos = Vector(*self.velocity) + self.pos
        
class PongGame(Widget):
    pass

class PongApp(App):
    def build(self):
        return PongGame()
    
if __name__ == '__main__':
    PongApp().run()

And here is the kv rule (to add to the kv file) used to style `PongBall` and to draw the ball as a white circle:

```python
<PongBall>:
    size: 50, 50
    canvas:
        Ellipse:
            pos: self.pos
            size: self.size
```

Note below that we also want make `PongBall` a child widget of the `PongGame` widget rule. We do this by adding `PongBall` to the `<PongGame>` block. 

In [ ]:
%%file pong.kv

#:kivy 1.0.9

<PongBall>:
    size: 50, 50 
    canvas:
        Ellipse:
            pos: self.pos
            size: self.size          

<PongGame>:
    canvas:
        Rectangle:
            pos: self.center_x-5, 0
            size: 10, self.height
    
    Label:
        font_size: 70  
        center_x: root.width / 4
        top: root.top - 50
        text: "0"
        
    Label:
        font_size: 70  
        center_x: root.width * 3 / 4
        top: root.top - 50
        text: "0"
    
    PongBall:
        center: self.parent.center

Now try things again!

In [ ]:
!python main.py